# Run one robot for one day

Run an experiment with a single robot exploring an environment. The environment and the robot policy is specified in experiment runs. 

In [ ]:
import sys
sys.path.append("..")
from settings import Config
import pathlib
from pprint import pprint
import gzip as compress
import copy
import pickle
from WaterberryFarm import WaterberryFarm, MiniberryFarm, WaterberryFarmEnvironment

from WbfExperiment import simulate_1day
from wbf_helper import get_geometry, create_wbfe
from WaterberryFarm import WaterberryFarm, MiniberryFarm, WBF_IM_DiskEstimator, WBF_IM_GaussianProcess, WBF_Score_WeightedAsymmetric, get_datadir

from Policy import FollowPathPolicy, RandomWaypointPolicy
from Robot import Robot
from WbfFigures import graph_env_im, graph_scores, hook_create_pictures, end_of_day_graphs


In [ ]:

# the experiment for the robot run
experiment = "1robot1day"
run = "random-waypoint-miniberry-30"
exp = Config().get_experiment(experiment, run)
pprint(exp)

# the exp for the environment
exp_env = Config().get_experiment("environment", exp["exp_environment"])
pprint(exp_env)
# the exp for policy
exp_policy = Config().get_experiment("policy", exp["exp_policy"])
pprint(exp_policy)


In [ ]:
def create_policy(exp_policy, exp_env):
    """Generate a policy, based on the specification of the policy and the environment"""
    if exp_policy["policycode"] == "RandomWaypointPolicy":
        geo = get_geometry(exp_env["typename"])
        # a random waypoint policy
        # geo = copy.copy(geom)
        policy = RandomWaypointPolicy(
            vel = 1, low_point = [geo["xmin"], 
            geo["ymin"]], high_point = [geo["xmax"], geo["ymax"]], seed = exp_policy["seed"])  
        policy.name = exp_policy["policyname"]
        return policy
    raise Exception(f"Unsupported policy type {exp_policy['policy_code']}")


In [ ]:

picture_paths = []

choices = {}
policy = create_policy(exp_policy, exp_env)
choices["policy-code"] = policy
choices["policy-name"] = policy.name

# these are the ones that need to be exp/run-ized 
choices["estimator-name"] = "AD"
geo = get_geometry(exp_env["typename"])
choices["estimator-code"] = WBF_IM_DiskEstimator(geo["width"], geo["height"])
choices["score-code"] = WBF_Score_WeightedAsymmetric()
choices["results_path"] = pathlib.Path(exp["data_dir"], "results")    

if choices["results_path"].exists():
    with compress.open(choices["results_path"], "rb") as f:
        results = pickle.load(f)
else:
    choices["velocity"] = exp["velocity"]
    choices["timesteps-per-day-override"] = exp["timesteps-per-day-override"]
    choices["time-start-environment"] = exp["time-start-environment"]
    choices["im_resolution"] = exp["im_resolution"]
    choices["results-basedir"] = exp["data_dir"]
    choices["action"] = "run-one-day"
    if "im_resolution" not in choices:
        choices["im_resolution"] = 1    
    choices["typename"] = exp_env["typename"]
    wbf, wbfe = create_wbfe(exp_env)
    # move ahead to the starting point of the environment
    wbfe.proceed(choices["time-start-environment"])
    choices["wbf"] = wbf
    choices["wbfe"] = wbfe
    # results["savedir"] = exp["data_dir"]
    #results["exp-name"] = results["typename"] + "_" # name of the exp., also dir to save data
    choices["days"] = 1
    #results["exp-name"] = results["exp-name"] + "1M_"
    get_geometry(choices["typename"], choices)
    # override the velocity and the timesteps per day, if specified
    if "timesteps-per-day-override" in choices:
        choices["timesteps-per-day"] = choices["timesteps-per-day-override"]
    if "velocity-override" in choices:
        choices["velocity"] = choices["velocity-override"]
    choices["robot"] = Robot("Rob", 0, 0, 0, env=None, im=None)
    # Setting the policy
    choices["robot"].assign_policy(choices["policy-code"])
    # 
    # This is where we actually calling the simulation
    #
    results = copy.deepcopy(choices)
    simulate_1day(results)
    print(f"Saving results to: {results['results_path']}")
    with compress.open(results["results_path"], "wb") as f:
        pickle.dump(results, f)
    


In [ ]:
if "picture-path" in results:
    print("Picture path in results!")
    picture_paths.append(results["picture-path"])

end_of_day_graphs(results, "daily-summary.pdf", plot_uncertainty = False)